In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_raw = pd.read_csv('data/dirty_cafe_sales.csv', sep=';')

In [3]:
item_price_mapping = {
    'Coffee': 2.0,
    'Tea': 1.5,
    'Sandwich': 4.5, #
    'Salad': 5.0,
    'Cake': 3.5, #
    'Cookie': 1.0,
    'Smoothie': 4.0,
    'Juice': 3.0
}

In [4]:
df_raw.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1005331,Coffee,1,2,2,Digital Wallet,Takeaway,04/11/23
1,TXN_1005472,Coffee,4,2,8,Credit Card,NaN,21/04/23
2,TXN_1016246,Coffee,1,2,2,ERROR,NaN,19/01/23
3,TXN_1020478,Coffee,1,2,2,Digital Wallet,Takeaway,09/03/23
4,TXN_1040764,Coffee,3,2,6,Cash,Takeaway,27/07/23


In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9822 non-null   object
 4   Total Spent       9850 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


## Data Cleaning ##

In [6]:
df = df_raw.copy()
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       178
Total Spent          150
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

### 1. Item ### 

**1.1 Handling Missing Values**

In [7]:
df[df['Item'].isnull()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
7397,TXN_1054915,NaN,5,1,5,NaN,In-store,22/01/23
7398,TXN_1124900,NaN,4,4,16,Credit Card,In-store,08/09/23
7399,TXN_1165762,NaN,3,2,6,Credit Card,NaN,22/10/23
7400,TXN_1166001,NaN,UNKNOWN,3,15,Cash,ERROR,NaN
7401,TXN_1205610,NaN,2,1,2,NaN,In-store,19/07/23
...,...,...,...,...,...,...,...,...
7725,TXN_9774251,NaN,2,3,6,Digital Wallet,Takeaway,16/06/23
7726,TXN_9810581,NaN,1,3,3,Cash,NaN,20/11/23
7727,TXN_9817602,NaN,2,4,8,Credit Card,Takeaway,26/12/23
7728,TXN_9945729,NaN,2,5,10,Digital Wallet,NaN,03/02/23


The NaN values in 'Item' still seem to have a corresponding 'Price Per Unit.' This 'Price Per Unit' can be used as a clue to link back to the 'Item Name' using a dictionary called <b>item_price_mapping</b>

In [8]:
# Reverse the structure mapping of item_price_mapping, into {price: item_name}
price_item_mapping = {price: item for item, price in item_price_mapping.items()}
price_item_mapping

{2.0: 'Coffee',
 1.5: 'Tea',
 4.5: 'Sandwich',
 5.0: 'Salad',
 3.5: 'Cake',
 1.0: 'Cookie',
 4.0: 'Smoothie',
 3.0: 'Juice'}

In [9]:
df['Item'].value_counts()

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
ERROR        292
Name: count, dtype: int64

In [10]:
# Fill the NaN of Item with Mapping price_item_mapping
df['Item'] = df['Item'].fillna(df['Price Per Unit'].map(price_item_mapping))

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9822 non-null   object
 4   Total Spent       9850 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


Turns out the total non-null count for Item <b>did not change</b>, meaning the .fillna() method did not affect the values. Looking back at the data type of 'Price Per Unit', which is still an object, this might be the root cause. Let's check the Price Per Unit value counts, but before that, let's replace df with df_raw.

In [12]:
df = df_raw.copy()
df['Price Per Unit'].value_counts()

Price Per Unit
3          1344
4          1249
2          1227
5          1204
1          1143
1.5        1133
3.5        1085
4.5        1083
ERROR       190
UNKNOWN     164
Name: count, dtype: int64

There are two suspicious values (ERROR and UNKNOWN) among those numeric values. This column should be of type float, as it represents a price. Let's handle those values before converting it to float.

In [13]:
df['Price Per Unit'] = df['Price Per Unit'].replace(['ERROR', 'UNKNOWN'], np.nan)
df['Price Per Unit'].value_counts()

Price Per Unit
3      1344
4      1249
2      1227
5      1204
1      1143
1.5    1133
3.5    1085
4.5    1083
Name: count, dtype: int64

In [14]:
df['Price Per Unit'] = df['Price Per Unit'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              9667 non-null   object 
 2   Quantity          9862 non-null   object 
 3   Price Per Unit    9468 non-null   float64
 4   Total Spent       9850 non-null   object 
 5   Payment Method    7421 non-null   object 
 6   Location          6735 non-null   object 
 7   Transaction Date  9841 non-null   object 
dtypes: float64(1), object(7)
memory usage: 625.1+ KB


Based on above information, the Price Per Unit are successfully converted into float64. Now let's get back to handling Missing Value on <b>Item</b>

In [15]:
# Fill the NaN of Item with Mapping price_item_mapping
df['Item'] = df['Item'].fillna(df['Price Per Unit'].map(price_item_mapping))
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1005331,Coffee,1,2.0,2,Digital Wallet,Takeaway,04/11/23
1,TXN_1005472,Coffee,4,2.0,8,Credit Card,NaN,21/04/23
2,TXN_1016246,Coffee,1,2.0,2,ERROR,NaN,19/01/23
3,TXN_1020478,Coffee,1,2.0,2,Digital Wallet,Takeaway,09/03/23
4,TXN_1040764,Coffee,3,2.0,6,Cash,Takeaway,27/07/23


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              9977 non-null   object 
 2   Quantity          9862 non-null   object 
 3   Price Per Unit    9468 non-null   float64
 4   Total Spent       9850 non-null   object 
 5   Payment Method    7421 non-null   object 
 6   Location          6735 non-null   object 
 7   Transaction Date  9841 non-null   object 
dtypes: float64(1), object(7)
memory usage: 625.1+ KB


The total non-null values in the 'Item' column have increased from 9,667 to 9,977. Let's examine the remaining missing values:

In [17]:
df.isnull().sum()

Transaction ID         0
Item                  23
Quantity             138
Price Per Unit       532
Total Spent          150
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

In [18]:
df[df['Item'].isnull()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
7402,TXN_1208561,NaN,ERROR,NaN,20,Credit Card,NaN,19/08/23
7448,TXN_2523298,NaN,4,NaN,6,ERROR,In-store,25/03/23
7449,TXN_2536573,NaN,2,NaN,8,Cash,In-store,24/06/23
7473,TXN_3334632,NaN,1,NaN,2,Credit Card,Takeaway,20/11/23
7480,TXN_3495950,NaN,4,NaN,6,Credit Card,In-store,19/02/23
7486,TXN_3611851,NaN,4,NaN,ERROR,Credit Card,NaN,09/02/23
7500,TXN_3803063,NaN,4,NaN,12,Credit Card,Takeaway,23/11/23
7509,TXN_4031509,NaN,4,NaN,16,Credit Card,Takeaway,04/01/23
7514,TXN_4208919,NaN,3,NaN,12,NaN,Takeaway,30/05/23
7543,TXN_4844386,NaN,5,NaN,15,Credit Card,In-store,28/10/23


The remaining missing values in the 'Item' column occurred because of <b>NaN values in 'Price Per Unit,'</b> which makes sense.

We might to have a different approach on finding the Item Name by the <b>Total Spent and Quantity</b> then get the price per unit to finally able to linked back to Item Name. Let's check value for those column Total Spent and Quantity

In [19]:
df['Quantity'].value_counts()

Quantity
5          2013
2          1974
4          1863
3          1849
1          1822
UNKNOWN     171
ERROR       170
Name: count, dtype: int64

In [20]:
df['Total Spent'].value_counts()

Total Spent
6          752
3          719
4          713
10         524
20         507
12         505
9          498
2          497
15         490
8          475
5          468
4.5        446
25         259
16         256
22.5       253
17.5       244
7.5        237
1          232
13.5       232
7          227
3.5        211
14         207
1.5        205
18         200
10.5       198
UNKNOWN    146
ERROR      146
0            3
Name: count, dtype: int64

On those columns each have two suspicious values called (UNKNOWN, ERROR) that need to be replaced with NaN

In [21]:
df['Quantity'] = df['Quantity'].replace(['ERROR', 'UNKNOWN'], np.nan)
df['Quantity'] = df['Quantity'].astype(float)
df['Total Spent'] = df['Total Spent'].replace(['ERROR', 'UNKNOWN'], np.nan)
df['Total Spent'] = df['Total Spent'].astype(float)

In [22]:
df['Quantity'].value_counts()

Quantity
5.0    2013
2.0    1974
4.0    1863
3.0    1849
1.0    1822
Name: count, dtype: int64

In [23]:
df['Total Spent'].value_counts()

Total Spent
6.0     752
3.0     719
4.0     713
10.0    524
20.0    507
12.0    505
9.0     498
2.0     497
15.0    490
8.0     475
5.0     468
4.5     446
25.0    259
16.0    256
22.5    253
17.5    244
7.5     237
13.5    232
1.0     232
7.0     227
3.5     211
14.0    207
1.5     205
18.0    200
10.5    198
0.0       3
Name: count, dtype: int64

After ensuring there are no more invalid values in the 'Total Spent' and 'Quantity' columns and successfully converting them to float, the 'Price Per Unit' values can now be replaced using Total Spent / Quantity.

In [24]:
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Total Spent']/df['Quantity'])

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              9977 non-null   object 
 2   Quantity          9521 non-null   float64
 3   Price Per Unit    9965 non-null   float64
 4   Total Spent       9558 non-null   float64
 5   Payment Method    7421 non-null   object 
 6   Location          6735 non-null   object 
 7   Transaction Date  9841 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


The total non-null values in 'Price Per Unit' have increased compared to last time, which is a good signal to conduct re-mapping for 'Item'.

In [26]:
df['Item'] = df['Item'].fillna(df['Price Per Unit'].map(price_item_mapping))

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              9998 non-null   object 
 2   Quantity          9521 non-null   float64
 3   Price Per Unit    9965 non-null   float64
 4   Total Spent       9558 non-null   float64
 5   Payment Method    7421 non-null   object 
 6   Location          6735 non-null   object 
 7   Transaction Date  9841 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


In [28]:
df[df['Item'].isnull()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
7402,TXN_1208561,NaN,NaN,NaN,20.0,Credit Card,NaN,19/08/23
7486,TXN_3611851,NaN,4.0,NaN,NaN,Credit Card,NaN,09/02/23


The data still has some missing 'Item' values. Since there are no more clues, it is better to categorize them as 'Others'.

In [29]:
df['Item'] = df['Item'].fillna('Others')

**1.2 Handling Inappropriate Values**

In [30]:
df['Item'].value_counts()

Item
Juice       1256
Coffee      1208
Salad       1186
Smoothie    1185
Cake        1139
Cookie      1132
Sandwich    1131
Tea         1125
UNKNOWN      344
ERROR        292
Others         2
Name: count, dtype: int64

Apparently there are two suspicious Item Name which are UNKNOWN and ERROR. Let's try to handle it by utilizing price_item_mapping dictioanry

In [31]:
df[np.logical_or(df['Item'] == 'UNKNOWN', df['Item'] == 'ERROR')]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
2257,TXN_1082717,ERROR,NaN,NaN,9.0,Digital Wallet,In-store,13/12/23
2258,TXN_1118799,ERROR,4.0,4.0,NaN,Credit Card,Takeaway,19/09/23
2259,TXN_1124796,ERROR,5.0,1.5,7.5,Credit Card,NaN,28/10/23
2260,TXN_1135761,ERROR,3.0,3.0,9.0,Digital Wallet,NaN,30/01/23
2261,TXN_1185003,ERROR,5.0,3.0,15.0,Cash,Takeaway,06/01/23
...,...,...,...,...,...,...,...,...
7392,TXN_9894204,UNKNOWN,2.0,2.0,4.0,Cash,Takeaway,08/08/23
7393,TXN_9904042,UNKNOWN,4.0,4.0,16.0,Credit Card,NaN,09/06/23
7394,TXN_9914084,UNKNOWN,1.0,2.0,2.0,ERROR,Takeaway,16/06/23
7395,TXN_9950775,UNKNOWN,4.0,5.0,20.0,Credit Card,Takeaway,20/02/23


In [32]:
df['Item'] = df.apply(lambda row: price_item_mapping.get(row['Price Per Unit'], row['Item'])
                      if row['Item'] in ['UNKNOWN', 'ERROR'] else row['Item'], axis=1)
df['Item'].value_counts()

Item
Juice       1418
Smoothie    1323
Coffee      1291
Salad       1272
Cookie      1213
Tea         1207
Cake        1139
Sandwich    1131
ERROR          2
UNKNOWN        2
Others         2
Name: count, dtype: int64

Apparently, the dataframe still has four rows with ['ERROR', 'UNKNOWN'] values in 'Item'. Let’s categorize them as 'Others' as well.

In [33]:
df['Item'] = df['Item'].replace(['ERROR', 'UNKNOWN'],'Others')
df['Item'].value_counts()

Item
Juice       1418
Smoothie    1323
Coffee      1291
Salad       1272
Cookie      1213
Tea         1207
Cake        1139
Sandwich    1131
Others         6
Name: count, dtype: int64

### 2. Transaction Date ### 

**2.1 Handling Missing Values**

In [34]:
df[df['Transaction Date'].isnull()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
26,TXN_1193930,Coffee,1.0,2.0,2.0,Cash,NaN,NaN
68,TXN_1529904,Coffee,1.0,2.0,2.0,Credit Card,In-store,NaN
73,TXN_1568657,Coffee,2.0,2.0,4.0,Cash,NaN,NaN
223,TXN_2574545,Coffee,NaN,2.0,4.0,NaN,In-store,NaN
226,TXN_2607233,Coffee,3.0,2.0,6.0,Cash,Takeaway,NaN
...,...,...,...,...,...,...,...,...
9717,TXN_4030010,Sandwich,5.0,4.5,22.5,NaN,Takeaway,NaN
9725,TXN_2376525,Sandwich,5.0,4.5,22.5,ERROR,NaN,NaN
9793,TXN_4620459,Sandwich,5.0,4.5,22.5,Cash,ERROR,NaN
9826,TXN_7133645,Sandwich,5.0,4.5,22.5,Credit Card,Takeaway,NaN


The dataframe has 159 rows missing 'Transaction Date', since this column is mandatory and can not be null then better to drop those rows rather than keep it. 

In [36]:
df.dropna(subset=['Transaction Date'], inplace=True)
df[df['Transaction Date'].isnull()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9841 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9841 non-null   object 
 1   Item              9841 non-null   object 
 2   Quantity          9371 non-null   float64
 3   Price Per Unit    9808 non-null   float64
 4   Total Spent       9408 non-null   float64
 5   Payment Method    7310 non-null   object 
 6   Location          6630 non-null   object 
 7   Transaction Date  9841 non-null   object 
dtypes: float64(3), object(5)
memory usage: 691.9+ KB


**2.2 Handling Invalid Values**

Before converting data type of 'Transaction Date' into date, better to handle the invalid value and drop them. 

In [44]:
idx_invalid_trxdate = df[np.logical_or(df['Transaction Date']=='ERROR', df['Transaction Date']=='UNKNOWN')].index
df.drop(idx_invalid_trxdate, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9540 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9540 non-null   object 
 1   Item              9540 non-null   object 
 2   Quantity          9086 non-null   float64
 3   Price Per Unit    9508 non-null   float64
 4   Total Spent       9120 non-null   float64
 5   Payment Method    7101 non-null   object 
 6   Location          6422 non-null   object 
 7   Transaction Date  9540 non-null   object 
dtypes: float64(3), object(5)
memory usage: 670.8+ KB


In [45]:
# Convert data type into datetime
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df.info()

/var/folders/lg/fsm8s9v909b1k6v2b54vczfw0000gn/T/ipykernel_83607/2254297040.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])


In [47]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1005331,Coffee,1.0,2.0,2.0,Digital Wallet,Takeaway,2023-04-11
1,TXN_1005472,Coffee,4.0,2.0,8.0,Credit Card,NaN,2023-04-21
2,TXN_1016246,Coffee,1.0,2.0,2.0,ERROR,NaN,2023-01-19
3,TXN_1020478,Coffee,1.0,2.0,2.0,Digital Wallet,Takeaway,2023-09-03
4,TXN_1040764,Coffee,3.0,2.0,6.0,Cash,Takeaway,2023-07-27
